# モジュールのインストール

In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install requests
!pip install BeautifulSoup4
!pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### 各ページの情報を持ってくる関数

In [14]:
def top_page_info(soup_page, find_tag, find_class):
    element = soup_page.find(find_tag, attrs = { "class" : find_class })
    if element is not None:
        data = element.text
        return data
    else:
        data = ""
        return data
    
#2行までしか対応してないよ！
def get_table_data(table_soup, dictionary):
    for row in table_soup.find_all("tr"):
        # 列中の全てのデータをパチパチと取ってきて一列の配列にする
        data_head = row.find_all("th")
        data_values = row.find_all("td")

        data_head_01 = data_head[0].text
        data_value_01 = data_values[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

        #　後半3行が1列しかないのでIndexOutOfRangeにならないように例外処理
        try:
            if data_head[1] is not None:
                data_head_02 = data_head[1].text
                data_value_02 = data_values[1].text.replace('\n', '').replace('\r', '').replace('\t', '')
                #dictionaryへの追加
                dictionary[data_head_01] = data_value_01
                dictionary[data_head_02] = data_value_02
        except IndexError:
            #dictionaryへの追加
            dictionary[data_head_01] = data_value_01
            
    return dictionary

In [15]:
# 各ページの情報を取ってくる関数
def get_page_info(urls, dataList):
    options = Options()
    options.add_argument('--headless')
    page_browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    for url in urls:
        url = "https://suumo.jp" + url.attrs["href"]
        # 出力用の辞書
        output_dict = {}

        #渡された引数のページを開きにいく
        page_browser.get(url)
        res_page = requests.get(url)
        soup_page = BeautifulSoup(res_page.content, "html.parser")

        # 家賃などの支払い情報の取得
        name = top_page_info(soup_page, "h1", "section_h1-header-title")
        rent = top_page_info(soup_page, "span", "property_view_note-emphasis")

        # 家賃などの支払い情報のテーブル取得
        payment_table = soup_page.find("div", attrs = {"class":"property_view_note-list"})
        payment_list = []
        if payment_table is not None:
            for data in payment_table.find_all("span"):
                payment_list.append(data.text)
        else:
            payment_data = ""
            payment_list.append(payment_data)
            print(name)

        # 物件外観を取得しに行く
        property_table = soup_page.find("table", attrs = { "class" : "property_view_table" })
        output_property = {}
        
        if property_table is not None:
            get_table_data(property_table, output_property)
            
        # 部屋の特徴・設備の取得
        characteristic_box = soup_page.find("div", attrs = { "class" : "bgc-wht" })
        characteristic_lis = characteristic_box.select("ul.inline_list > li")
        for li in characteristic_lis:
            #rooms_characteristicに部屋の特徴と設備が入ってくる         
            rooms_characteristic = li.text

        # 物件概要を取得しに行く
        # 物件情報のテーブルを全て取得する部分    
        equipment_table = soup_page.find("table", attrs = { "class" : "table_gaiyou" })
        output_row = {}
        get_table_data(equipment_table, output_row)
        

        # 出力用に加工する
        output_dict = { "name" : name, "rent": rent, "payment" : payment_list, "outline" : output_property, "characteristic" : rooms_characteristic, "equipment" : output_row}

        dataList.append(output_dict)
        time.sleep(1)
    page_browser.quit()

## 全てのページの情報を持ってきてみようと試みる

In [16]:
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=020&bs=040&ta=06&sc=06201&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2="

# 一度ブラウザ起動
options = Options()
options.add_argument('--headless')
# browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)

count = 0

# parametarの宣言
dataList = []

while True:
        url = browser.current_url
        print(url)
        # 一度ブラウザ起動
        options = Options()
        options.add_argument('--headless')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        browser.get(url)

        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            elems = soup.find_all("a", class_="js-cassette_link_href") 
            get_page_info(elems, dataList)
                
            browser.find_element_by_link_text('次へ').click()
            if count % 10 == 0:
                print("pages:{0}".format(count))
                print('==============================================')
                time.sleep(10)
            time.sleep(10)
            count += 1
            
        except NoSuchElementException:
            browser.quit()
            break



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=020&bs=040&ta=06&sc=06201&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


{'所在地': '山形県山形市城西町５', '駅徒歩': 'ＪＲ仙山線/北山形駅 歩30分ＪＲ奥羽本線/山形駅 歩31分山交バス/西小学校口 歩5分', '間取り': '2LDK', '専有面積': '55.79m2', '築年数': '築21年', '階': '2階', '向き': '南', '建物種別': 'アパート'}
{'間取り詳細': '洋8 洋8 LDK11.3', '構造': '鉄筋コン', '階建': '2階/3階建', '築年月': '2001年3月', '損保': '1.7万円2年', '駐車場': '付無料/駐2台可', '入居': '即', '取引態様': '仲介', '条件': '-', '取り扱い店舗物件コード': '-', 'SUUMO物件コード': '100122897575', '総戸数': '12戸', '情報更新日': '2021/06/24', '次回更新日': '次回更新日は情報更新日より１５日以内', '仲介手数料': '1.1ヶ月', '保証会社': 'オリコ利用可 契約時初回保証料：賃料の50％', 'ほか諸費用': '家財保険700円～月々払い可/安心入居さポート8800円（任意）'}
{'所在地': '山形県山形市陣場南', '駅徒歩': 'ＪＲ左沢線/東金井駅 歩9分ＪＲ仙山線/北山形駅 歩36分ＪＲ仙山線/羽前千歳駅 歩52分', '間取り': '1LDK', '専有面積': '43.21m2', '築年数': '新築', '階': '1階', '向き': '南東', '建物種別': 'アパート'}
{'間取り詳細': '洋5.8 LDK11.7', '構造': '木造', '階建': '1階/2階建', '築年月': '2021年9月', '損保': '1.5万円2年', '駐車場': '敷地内5500円/駐2台可', '入居': "'21年10月上旬", '取引態様': '仲介', '条件': 'ペット相談', '取り扱い店舗物件コード': '-', 'SUUMO物件コード': '100238460999', '総戸数': '-', '情報更新日': '2021/06/17', '次回更新日': '次回更新日は情報更新日より８日以内', '仲介手数料': '1.1ヶ月', '保証会社': '保証会社利用必 



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


...

{'所在地': '山形県山形市南原町１', '駅徒歩': 'ＪＲ仙山線/山形駅 歩28分ＪＲ仙山線/北山形駅 歩54分', '間取り': '1K', '専有面積': '20.35m2', '築年数': '築26年', '階': '2階', '向き': '東', '建物種別': 'マンション'}
{'間取り詳細': '洋7', '構造': '鉄骨', '階建': '2階/4階建', '築年月': '1996年4月', '損保': '1.2万円2年', '駐車場': '敷地内6000円/屋根付駐', '入居': "'21年8月下旬", '取引態様': '仲介', '条件': '-', '取り扱い店舗物件コード': '-', 'SUUMO物件コード': '100212002769', '総戸数': '33戸', '情報更新日': '2021/06/24', '次回更新日': '次回更新日は情報更新日より１５日以内', '契約期間': '普通借家 2年', '仲介手数料': '1.1ヶ月', 'ほか諸費用': '町内会費:500円/月', '備考': '巡回管理/リフォーム：2019年3月全館リノベーション'}
{'所在地': '山形県山形市城北町１', '駅徒歩': 'ＪＲ奥羽本線/山形駅 歩23分ＪＲ奥羽本線/北山形駅 歩15分ＪＲ左沢線/東金井駅 歩43分', '間取り': '1K', '専有面積': '28.89m2', '築年数': '築27年', '階': '2階', '向き': '南', '建物種別': 'アパート'}
{'間取り詳細': '洋7 ロフト 6', '構造': '軽量鉄骨', '階建': '2階/2階建', '築年月': '1995年5月', '損保': '1.7万円2年', '駐車場': '敷地内5000円', '入居': '即', '取引態様': '仲介', '条件': '事務所利用不可', '取り扱い店舗物件コード': '1656652', 'SUUMO物件コード': '100242613739', '総戸数': '16戸', '情報更新日': '2021/06/23', '次回更新日': '次回更新日は情報更新日より１５日以内', '契約期間': '普通借家 2年', '仲介手数料': '1.1ヶ月', '保証会社': '保証会社利用必 



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472


https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=020&bs=040&ta=06&sc=06201&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&page=3


Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


KeyboardInterrupt: 

In [5]:
print(len(dataList))

3722


In [6]:
df = pd.DataFrame(dataList)
# #カラムの並び替え
# df.reindex(columns=['postDay', 'point', ' when', 'congestion', 'tag', 'numberOfPeople', 'title', 'text'])
# df.head().append(df.tail())
df.to_csv('{filename}.csv'.format(filename = "YamagataCity"))
# elapsed_time = time.time() - startTime
# print(elapsed_time)

In [ ]:
# 各ページの情報を取ってくる関数
def get_page_info(urls, dataList):
    options = Options()
    options.add_argument('--headless')
    page_browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    for url in urls:
        url = "https://suumo.jp" + url.attrs["href"]
        # 出力用の辞書
        output_dict = {}

        #渡された引数のページを開きにいく
        page_browser.get(url)
        res_page = requests.get(url)
        soup_page = BeautifulSoup(res_page.content, "html.parser")

        # 家賃などの支払い情報の取得
        if soup_page.find("h1", attrs = { "class" : "section_h1-header-title" }) is not None:
            name = soup_page.find("h1", attrs = { "class" : "section_h1-header-title" }).text
        else:
            name = ""
#         name = top_page_info(soup_page, "h1", "section_h1-header-title")
        
        if soup_page.find("span", attrs = { "class" : "property_view_note-emphasis" }) is not None:
            rent = soup_page.find("span", attrs = { "class" : "property_view_note-emphasis" }).text
        else:
            rent = ""
            print(name)
#         rent = top_page_info(soup_page, "span", "property_view_note-emphasis")

        payment_table = soup_page.find("div", attrs = {"class":"property_view_note-list"})
        payment_list = []
        if payment_table is not None:
#             payment_data = soup_page.find("div", attrs = {"class":"property_view_note-list"})
            for data in payment_table.find_all("span"):
                payment_list.append(data.text)
        else:
            payment_data = ""
            payment_list.append(payment_data)
            print(name)

        # 物件外観を取得しに行く
        property_table = soup_page.find("table", attrs = { "class" : "property_view_table" })
        output_property = {}
        
        if property_table is not None:
            property_table_rows = property_table.find_all("tr")
            for row in property_table.find_all("tr"):
                # 列中の全てのデータをパチパチと取ってきて一列の配列にする
                data_property_head = row.find_all("th")
                data_property_values = row.find_all("td")

                data_property_head_01 = data_property_head[0].text
                data_property_value_01 = data_property_values[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

                #　後半3行が1列しかないのでIndexOutOfRangeにならないように例外処理
                try:
                    if data_property_head[1] is not None:
                        data_property_head_02 = data_property_head[1].text
                        data_property_value_02 = data_property_values[1].text.replace('\n', '').replace('\r', '').replace('\t', '')

                        output_property[data_property_head_01] = data_property_value_01
                        output_property[data_property_head_02] = data_property_value_02
                except IndexError:
                    #dictionaryへの追加
                    output_property[data_property_head_01] = data_property_value_01

        # 部屋の特徴・設備の取得
        characteristic_box = soup_page.find("div", attrs = { "class" : "bgc-wht" })
        characteristic_lis = characteristic_box.select("ul.inline_list > li")
        for li in characteristic_lis:
            #rooms_characteristicに部屋の特徴と設備が入ってくる         
            rooms_characteristic = li.text

        # 物件概要を取得しに行く
        equipment_table = soup_page.find("table", attrs = { "class" : "table_gaiyou" })
        equipment_table_rows = equipment_table.find_all("tr")
        # 物件情報のテーブルを全て取得する部分    
        output_row = {}
        for row in equipment_table.find_all("tr"):
            # 列中の全てのデータをパチパチと取ってきて一列の配列にする
            data_equipment_head = row.find_all("th")
            data_equipment_values = row.find_all("td")

            data_equipment_head_01 = data_equipment_head[0].text
            data_equipment_value_01 = data_equipment_values[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

            #　後半3行が1列しかないのでIndexOutOfRangeにならないように例外処理
            try:
                if data_equipment_head[1] is not None:
                    data_equipment_head_02 = data_equipment_head[1].text
                    data_equipment_value_02 = data_equipment_values[1].text.replace('\n', '').replace('\r', '').replace('\t', '')
                    #dictionaryへの追加
                    output_row[data_equipment_head_01] = data_equipment_value_01
                    output_row[data_equipment_head_02] = data_equipment_value_02
            except IndexError:
                #dictionaryへの追加
                output_row[data_equipment_head_01] = data_equipment_value_01

        # 出力用に加工する
        output_dict = { "name" : name, "rent": rent, "payment" : payment_list, "outline" : output_property, "characteristic" : rooms_characteristic, "equipment" : output_row}

        dataList.append(output_dict)
        time.sleep(1)
    page_browser.quit()